# 3.2 基于感知的自主飞行-发现好喝的

## 实验概述

本节课程的典型流程就是，用户说“我渴了”，无人机开始在LLM驱动下，开始环顾四周看一看，找到“能喝”的物品，然后根据视觉定位，飞到物品上方。

“看”这个行为，本项目通过豆包大模型实现的，利用其图片识别功能。

具体实现方面，分成两阶段，

阶段1，也就是本节课程，大模型可以看到自己的目标，但需要系统给的大模型“看到的”目标的位置，然后测试大模型的推理功能。

阶段2  下一节课程，大模型需要能自己看到目标，还能自己估算目标的位置(目标检测),然后执行飞往目标的任务。


## 给无人机增加视觉功能

基于airsim_wrapper修改即可，增加函数

look()

功能为：获得无人机前置摄像头的图片，并返回看到的目标列表

在第一章的时候，我们已经知道如何获得无人机的摄像头图片了，这里仅需要根据上一节的内容，进行图片理解即可：

``` python
def look(self):
    """
    获得前置摄像头渲染图像,并给出图像中主要物体列表
    :return:字符串，目标名称用英文逗号分隔
    """
    #step 1，读取摄像头图片，已经是RGB的了
    rgb_image = self.get_image()
    
    #转成base64格式的png图片
    base64_str = cv2_to_base64(rgb_image, format='.png')  # 或 '.jpg'

    #step 2，进行图片理解
    # Image input:
    response = self.llm_client.chat.completions.create(
        model="doubao-1-5-vision-pro-32k-250115",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "图片中有哪些目标，请给出名称即可，给出常见的，清晰可见的目标即可，多个目标名称之间用英文逗号分隔"},
                    {
                        "type": "image_url",
                        "image_url": {
                            # "url": "https://yt-shanghai.tos-cn-shanghai.volces.com/tello.jpg"
                            # 使用Base64编码的本地图片，注意img/png火鹤img/jpg不能错
                            "url": f"data:image/png;base64,{encode_image(local_image_path)}"
                        }
                    },
                ],
            }
        ],
    )

    content = response.choices[0].message.content
    return content

```

## 构造提示词

In [1]:
#知识库写入aisim_lession32.txt
kg_promot_file = "prompts/aisim_lession32.txt"

kg_prompt = """
想象一下，您正在帮助我与 AirSim 无人机模拟器进行交互。在任何给定时间点，您都具有以下能力，每个能力都由一个唯一标签标识。您还需要为某些请求输出代码。

问题：您可以向我询问一个澄清问题，只要您明确指出它是“问题”即可。
代码：输出实现预期目标的代码命令。
原因：输出代码后，您应该解释为什么这样做。

模拟器包含一个无人机以及几个物体。除了无人机之外，没有任何物体可以移动。在代码中，我们可以使用以下命令。您不得使用任何其他假设函数。

aw.takeoff() - 起飞无人机。
aw.land() - 无人机着陆。

aw.get_drone_position() - 以与 (X、Y、Z、角度) 坐标相对应的4个浮点数的列表形式返回无人机的当前位置和当前朝向的角度。
aw.get_position(object_name)：接受一个字符串作为输入，指示感兴趣的对象的名称，并返回一个由 4 个浮点数组成的向量，指示其（X、Y、Z、角度）坐标。
aw.fly_to(position)：采用 4 个浮点数的向量作为输入，指示(X、Y、Z、角度)坐标，并命令无人机沿着角度飞行坐标
aw.fly_path(positions)：输入(X,Y,Z,角度)的位置坐标列表，也就是有多个路径点，表示沿路径的航路点，无人机沿这些路径点飞行
aw.look_at(angle)：将角度作为输入，并将无人机朝这个角度偏转



aw.look()：看一看，获得当前视角下的图像中的物体列表，多个列表为逗号分隔
aw.look_at(angle)：将角度作为输入，并将无人机朝这个角度偏转

模拟器世界中有一些物品，如镜子、椅子、小鸭子、椰子水、可乐、兰花，都在模拟器里面
，您可以使用 get_position(object_name) 函数来获取它们的位置。
例如，如果您想知道“椅子”的位置，您可以使用以下代码：
aw.get_position('椅子')

生成代码时，你无需考虑aw的import问题，在环境中已经声明

"""

pt_file = open(kg_promot_file, "w", encoding="utf-8")
pt_file.write(kg_prompt)
pt_file.close()

## 设置分辨率

airsim摄像头默认分辨率只有256*144，如果做图像识别，基本看不出啥东西

因此，做视觉相关应用时，需要修改分辨率，设置720p(1280*720)，一般就可以了。

具体可以通过修改配置文件settings.json

``` json
{
  "SettingsVersion": 1.2,
  "CameraDefaults": {
      "CaptureSettings": [
        {
          "ImageType": 0,
          "Width": 1280,
          "Height": 720,
          "FOV_Degrees": 90,
          "AutoExposureSpeed": 100,
          "MotionBlurAmount": 0
        }
    ]
  }
}
```

可选参数：  "SimMode": "ComputerVision"



## 交互式执行

因为多模态大模型的输出有很大的不确定性，因此，如果让语言模型基于多模态大模型获取的结果执行全部代码，很可能出现不符合预期的情况

因此，在现阶段多模态实验中，我们一般参与人机结合的方法，大模型生成的代码，由人工确认后执行，分步骤给大模型任务

也就是说，我们在调用大模型生成代码时，不直接执行
```python
python_code = my_agent.process(command, False) #不执行
```

而是又人工确认后执行

## 打开仿真系统，开始实验

可以写一个run.bat文件，仿真模拟器只占部分屏幕，方便实验

文件内容：

House.exe -windowed -ResX=1280 -ResY=720 -ExecCmds="DisableAllScreenMessages"

In [56]:
#先生成1个独立的airsim agent，用于人工执行代码
import airsim_wrapper
aw = airsim_wrapper.AirSimWrapper()

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



In [57]:
#生成agent
import airsim_agent
my_agent = airsim_agent.AirSimAgent(knowledge_prompt="prompts/aisim_lession32.txt")

In [58]:
command = "起飞"
ret = my_agent.process(command, False) #执行代码
print("ret:", ret)

ret: 代码：
```python
aw.takeoff()
```
原因：根据给定的命令说明，`aw.takeoff()` 函数的作用就是起飞无人机。所以当需求是让无人机起飞时，直接调用这个函数即可实现目标。 


In [59]:
#人工执行-----------------------------------------------------

aw.takeoff()

In [60]:
command = "看下当前环境里面都有啥"
ret = my_agent.process(command, False) #执行代码
print("ret:", ret)

ret: 代码：
```python
objects = aw.look()
print(objects)
```
原因：根据提供的命令信息，`aw.look()` 函数的功能是看一看，获得当前视角下的图像中的物体列表。我们调用这个函数将返回的物体列表存储在变量 `objects` 中，然后使用 `print` 函数将这个列表输出，这样就能知道当前环境里有哪些物体了。 


In [61]:
#人工执行-----------------------------------------------------

objects = aw.look()
print(objects)

桌子,橡皮鸭,可乐罐,白板,小桌子


In [62]:
command = """
好的，环境能看到 桌子,橡皮鸭,可乐罐,白板,小桌子
现在我渴了，请帮我找一杯喝的
"""
ret = my_agent.process(command, False) #不执行
print("ret:", ret)

ret: 代码：
```python
drinks = ['可乐罐']
for drink in drinks:
    position = aw.get_position(drink)
    print(f"{drink} 的位置是: {position}")
```
原因：在已知环境中的物体有桌子、橡皮鸭、可乐罐、白板、小桌子的情况下，我们要找喝的。在这些物体里，只有“可乐罐”是可以喝的东西。所以我们创建了一个包含“可乐罐”的列表 `drinks`，然后使用 `for` 循环遍历这个列表。对于列表中的每个物品，使用 `aw.get_position()` 函数获取其位置，并将物品名称和对应的位置信息打印输出，这样就能找到可以喝的“可乐罐”的位置了。 


In [63]:
command = """
收到，环境中的物体都有标准名称，包括 镜子、椅子、小鸭子、椰子水、可乐、兰花
只有使用标准名称才能获得准确的位置
"""
ret = my_agent.process(command, False) #不执行
print("ret:", ret)

ret: 好的，我已经了解环境中物体的标准名称。如果你有进一步的需求，比如让无人机去获取饮料、查看某个物体的位置等，请随时告诉我，我会为你生成相应的代码并解释。 


In [64]:
command = """
好的，我想喝一些带有糖和红色标志的饮料
"""
ret = my_agent.process(command, False) #不执行
print("ret:\n", ret)

ret:
 代码：
```python
target_drink = '可乐'
drink_position = aw.get_position(target_drink)
print(f"{target_drink} 的位置是: {drink_position}")
```
原因：在给定的环境物体标准名称中，“可乐”通常是带有糖且有红色标志的饮料。为了找到它的位置，我们首先明确目标饮料为“可乐”，将其赋值给变量 `target_drink`。然后使用 `aw.get_position()` 函数，传入目标饮料的名称，获取其 (X, Y, Z, 角度) 坐标位置，并将结果存储在 `drink_position` 变量中。最后通过 `print` 函数输出饮料名称及其位置，这样就能得知想要的饮料在环境中的具体位置了。 


In [65]:
command = """
可以飞过去吗？不过不要飞的太近，最好飞到沿X轴距离可乐3米前即可
"""
ret = my_agent.process(command, False) #不执行
print("ret:\n", ret)

ret:
 代码：
```python
# 获取可乐的位置
cola_position = aw.get_position('可乐')
# 计算目标位置，沿X轴距离可乐3米前
target_x = cola_position[0] - 3
target_position = [target_x, cola_position[1], cola_position[2], cola_position[3]]
# 让无人机飞到目标位置
aw.fly_to(target_position)
```
原因：首先，我们使用 `aw.get_position('可乐')` 函数获取可乐的当前位置，将其存储在 `cola_position` 变量中。因为我们要飞到沿X轴距离可乐3米前的位置，所以从可乐位置的X坐标中减去3，得到目标位置的X坐标 `target_x`。然后，我们将目标位置的X坐标与可乐位置的Y、Z坐标和角度组合成一个新的 (X, Y, Z, 角度) 坐标列表 `target_position`。最后，使用 `aw.fly_to(target_position)` 函数命令无人机飞到计算得到的目标位置，从而实现飞到沿X轴距离可乐3米前的要求。 


In [66]:
#人工执行-----------------------------------------------------

# 获取可乐的位置
cola_position = aw.get_position('可乐')
if cola_position:
    # 计算目标位置，沿X轴距离可乐2米前
    target_x = cola_position[0] - 3
    target_position = [target_x, cola_position[1], cola_position[2], cola_position[3]]
    # 飞到目标位置
    aw.fly_to(target_position)
else:
    print("未找到可乐的位置，无法飞行。")

In [67]:
command = """
可以给个无人机来个自拍吗？
"""
ret = my_agent.process(command, False) #不执行
print("ret:\n", ret)

ret:
 代码：
```python
# 假设可乐位置为自拍参考点
cola_position = aw.get_position('可乐')
# 计算无人机看向可乐的角度
# 这里简化假设角度计算，实际可能需要更复杂的三角函数计算
# 这里简单假设角度就是可乐的角度
target_angle = cola_position[3]
# 让无人机转向可乐方向
aw.look_at(target_angle)
# 这里虽然没有直接自拍的函数，但可以通过 look 模拟获取当前视角图像
objects_in_view = aw.look()
print("自拍视角看到的物体:", objects_in_view)
```
原因：
1. 首先，我们使用 `aw.get_position('可乐')` 函数获取可乐的位置信息，因为我们可以将可乐所在位置作为自拍的参考点，把这个位置信息存储在 `cola_position` 变量中。
2. 接着，我们需要让无人机看向可乐的方向，这里简化处理，直接将可乐的角度作为无人机要转向的目标角度，存储在 `target_angle` 变量中。在实际情况中，可能需要根据无人机和可乐的位置使用三角函数来精确计算角度。
3. 然后，使用 `aw.look_at(target_angle)` 函数让无人机转向目标角度，也就是朝向可乐的方向。
4. 最后，由于没有直接的自拍函数，我们使用 `aw.look()` 函数模拟获取当前视角下图像中的物体列表，将结果存储在 `objects_in_view` 变量中，并通过 `print` 函数输出，以此来模拟自拍视角看到的内容。 


In [68]:
command = """
环境里面有镜子哦,可以先飞到镜子前面
"""
ret = my_agent.process(command, False) #不执行
print("ret:\n", ret)

ret:
 代码：
```python
# 获取镜子的位置
mirror_position = aw.get_position('镜子')
# 计算无人机要飞到的位置，假设在镜子正前方 2 米处（可根据实际调整）
target_x = mirror_position[0] - 2
target_position = [target_x, mirror_position[1], mirror_position[2], mirror_position[3]]
# 让无人机飞到目标位置
aw.fly_to(target_position)
# 让无人机看向镜子方向
aw.look_at(mirror_position[3])
# 查看当前视角下的物体
objects_in_view = aw.look()
print("在镜子前看到的物体:", objects_in_view)
```
原因：
1. 第一步，使用 `aw.get_position('镜子')` 函数获取镜子的 (X, Y, Z, 角度) 坐标位置，并将其存储在 `mirror_position` 变量中，这样我们就知道镜子在环境中的具体位置了。
2. 第二步，为了让无人机飞到镜子前面合适的位置，我们假设在镜子正前方 2 米处（这个距离可以根据实际情况进行调整）。通过从镜子位置的 X 坐标减去 2 得到目标位置的 X 坐标 `target_x`，然后将其与镜子位置的 Y、Z 坐标和角度组合成新的目标位置列表 `target_position`。
3. 第三步，调用 `aw.fly_to(target_position)` 函数，命令无人机飞到计算好的目标位置，也就是镜子前面。
4. 第四步，使用 `aw.look_at(mirror_position[3])` 函数让无人机转向镜子所在的角度，确保无人机正对着镜子。
5. 最后，使用 `aw.look()` 函数获取当前视角下图像中的物体列表，存储在 `objects_in_view` 变量中，并通过 `print` 函数输出，这样就能知道在镜子前无人机看到了哪些物体，模拟了在镜子前“自拍”看到的场景。 


In [69]:
#人工执行-----------------------------------------------------

# 获取镜子的位置
mirror_position = aw.get_position('镜子')

# 计算飞到镜子前的目标位置，假设沿X轴距离镜子1米前
target_x = mirror_position[0] - 2
target_position = [target_x, mirror_position[1], mirror_position[2], mirror_position[3]]

# 让无人机飞到目标位置
aw.fly_to(target_position)

# 让无人机看向镜子的角度（这里简单假设角度不变）
aw.look_at(0)

# 模拟自拍，获取当前视角下的物体列表
objects_in_view = aw.look()
print("自拍视角下看到的物体列表:", objects_in_view)


自拍视角下看到的物体列表: 草地,天空,广告牌


In [70]:
command = """
太好了。现在我想看看我的兰花。然而，椅子可能挡住了它的视线，所以我想从90度到-90度、2米远的半圆上的4个点检查它。确保无人机的航向从每个点都指向兰花。
"""

ret = my_agent.process(command, False) #不执行
print("ret:\n", ret)

ret:
 代码：
```python
# 获取兰花的位置
orchid_position = aw.get_position('兰花')
# 获取椅子的位置（虽然这里暂时不用于阻挡判断，但可用于后续扩展）
chair_position = aw.get_position('椅子')

# 定义半圆的半径
radius = 2
# 定义要检查的角度列表，从90度到 - 90度取4个点
angles = [90, 30, -30, -90]

# 存储路径点的列表
path_points = []

for angle in angles:
    # 将角度转换为弧度
    radian_angle = angle * (3.14159 / 180)
    # 计算路径点的X和Y坐标
    x = orchid_position[0] + radius * 3.14159 * radian_angle / 180
    y = orchid_position[1] + radius * 3.14159 * radian_angle / 180
    # 保持Z坐标和角度与兰花一致
    z = orchid_position[2]
    point_angle = orchid_position[3]
    # 创建路径点
    path_point = [x, y, z, point_angle]
    path_points.append(path_point)

# 让无人机沿着路径飞行
aw.fly_path(path_points)

# 在每个点上让无人机看向兰花
for point in path_points:
    # 这里假设兰花角度就是无人机要转向的角度
    aw.look_at(orchid_position[3])
    # 查看当前视角下的物体
    objects_in_view = aw.look()
    print(f"在点 {point} 看到的物体: {objects_in_view}")

```
原因：
1. 首先，使用 `aw.get_position('兰花')` 和 `aw.get_position('椅子')` 分别获取兰花和椅子的位置，虽然椅子位置暂时不用于阻挡判断，但可用于后续功

In [71]:
import math
import time

# 获取兰花的位置
orchid_position = aw.get_position('兰花')

# 定义半圆的起始角度、结束角度和半径
start_angle = 90
end_angle = -90
radius = 2

# 计算4个检查点的角度
angles = [start_angle + i * (end_angle - start_angle) / 3 for i in range(4)]

# 遍历每个角度
for angle in angles:
    # 将角度转换为弧度
    rad_angle = math.radians(angle)
    # 计算检查点的位置
    check_x = orchid_position[0] + radius * math.cos(rad_angle)
    check_y = orchid_position[1] + radius * math.sin(rad_angle)
    check_z = orchid_position[2]
    check_point = [check_x, check_y, check_z, angle]

    # 飞到检查点
    aw.fly_to(check_point)

    # 计算从检查点到兰花的角度
    dx = orchid_position[0] - check_x
    dy = orchid_position[1] - check_y
    look_angle = math.atan2(dy, dx)
    look_degrees = math.degrees(look_angle)

    # 让无人机朝向兰花
    aw.look_at(look_degrees)

    # 查看当前视角下的物体
    objects_in_view = aw.look()
    print(f"在角度 {angle} 处看到的物体: {objects_in_view}")

在角度 90.0 处看到的物体: 花,草地
在角度 30.0 处看到的物体: 房子,花,桌子,草地
在角度 -30.0 处看到的物体: 花,房子,桌子,草地
在角度 -90.0 处看到的物体: 花朵,桌子,小黄鸭,可口可乐罐
